# Unit 4 Lesson 1 Project 5

In [29]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

## Take the loan data and process it as you did previously to build your linear regression model.

In [30]:
# loansData = pd.read_csv('https://spark-public.s3.amazonaws.com/dataanalysis/loansData.csv')
loansData = pd.read_csv('loansData.csv')

loansData['FICO.Score'] = loansData['FICO.Range'].map(lambda x: int(x.split('-')[0]))

intrate = [float(ir.strip('%')) for ir in loansData['Interest.Rate']] # list
loanamt = loansData['Amount.Requested'] # Series
fico = loansData['FICO.Score'] # Series

## Break the data-set into 10 segments following the example provided here in KFold .

In [31]:
k = 10
n = len(loansData['Interest.Rate'])
kf = KFold(n, n_folds=k) 

## Compute each of the performance metric (MAE, MSE or R2) for all the folds. 

In [32]:
print(min(train), max(train))
print(min(test), max(test))


0 2249
2250 2499


In [44]:
mse, mas, r2 = 0, 0, 0

for train, test in kf:
    # TRAIN
    it_train = pd.Series(intrate).iloc[min(train):max(train)] # interest rate
    la_train = loanamt.iloc[min(train):max(train)] # loan amount
    fi_train = fico.iloc[min(train):max(train)] # fico

    x1_train = np.matrix(fi_train).transpose()     # fico is a series; transpose converts vertically
    x2_train = np.matrix(la_train).transpose()

    train_X = np.column_stack([x1_train,x2_train])
    train_y = np.matrix(it_train).transpose()

    X_train = sm.add_constant(train_X)

    model = sm.OLS(train_y,X_train)
    f = model.fit()   
    # print f.summary()    

    # TEST 
    it_test = pd.Series(intrate).iloc[min(test):max(test)]
    la_test = loanamt.iloc[min(test):max(test)]
    fi_test = fico.iloc[min(test):max(test)]

    x1_test = np.matrix(fi_test).transpose()
    x2_test = np.matrix(la_test).transpose()

    test_X = np.column_stack([x1_test,x2_test])

    X_test = sm.add_constant(test_X)    

    y_actual = test_y = np.matrix(intrate).transpose()[test[0]:test[-1]] # is this right?

    y_predicted = f.predict(X_test)
    
    mse += mean_squared_error(y_actual, y_predicted) 
    mas += mean_absolute_error(y_actual, y_predicted)
    r2  += r2_score(y_actual, y_predicted)

## ... The average would be the performance of your model.

In [46]:
print(float(mse)/len(kf), float(mas)/len(kf), float(r2)/len(kf))

6.002865721327119 1.9384998435815615 0.6541520318534143


## Comment on each of the performance metric you obtained.



    Mean Squared Error: The square of the error between what our model predicts for a given data point, and the actual value in the test set.

    Mean Absolute Error: The absolute value of the error between our predicted outcomes, and the actual response in the test set.

    R-Squared: Also known as the coefficient of determination, this number indicates how well our model fits the data.
